In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import butterfly.album
import butterfly.CNN
from itertools import combinations 
from joblib import parallel_backend
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score

In [17]:
DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
#DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [18]:
#Decide which dataset you want to predict
resp_n = 'plasma_l'
response = [col for col in DF if col.startswith(resp_n)]
response.append("patientID")
response.append("trimester")    
response_df = DF[response]

In [19]:
# Choose your responses, size, for the image and number of features we are predicting
pixels = 40
features = len(response)-2

In [20]:
#Get every combination of patient IDs in pairs of m's
# Function which returns subset or r length from n 
m = 2
IDs = response_df['patientID'].unique()
IDs = IDs.tolist()
exclude = list(combinations(IDs, m))

In [21]:
predictors = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']
predictors.remove(resp_n)

In [ ]:
albums = Parallel(n_jobs=3)(delayed(butterfly.album.create_album)
                                                (DF, predictors[al], pixels) 
                                                for al in tqdm(range(6)))

  0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
#Using one omics at a time
bars_p = []
bars_o = []

for pred in tqdm(range(6)):

    prediction, observed = zip(*Parallel(n_jobs=3)
                               (delayed(butterfly.CNN.model)
                                (response_df, exclude, cv, albums[pred], pixels, features) 
                                for cv in range(len(exclude))))
                
    bars_p.append(pd.concat(prediction))
    bars_o.append(pd.concat(observed))

100%|██████████| 6/6 [33:25<00:00, 334.27s/it]


In [11]:
#Choose your correlation statistic
statistic = 'spearman'

In [12]:
#Using all omics at once
prediction, observed = zip(*Parallel(n_jobs=3)(delayed(butterfly.CNN.model_multi)
                                               (response_df, exclude, cv, albums, pixels, features) 
                                               for cv in tqdm(range(len(exclude)))))

100%|██████████| 136/136 [16:23<00:00,  7.23s/it]


In [13]:
#All omics at once
pred = pd.concat(prediction)
obs  = pd.concat(observed)

In [14]:
#All omics at once
correl = pred.corrwith(obs, axis = 0, method = statistic) 
correl[correl < 0] = 0
results.append(np.mean(correl))

In [15]:
predictors.append('all')
y_pos = np.arange(len(predictors))